Imporing required librairies

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler

max_length = 20


color_pal = sns.color_palette()
plt.style.use('fivethirtyeight')



Reading the data

In [ ]:
csv_file_path = 'forecast_data 1.csv'
df = pd.read_csv(csv_file_path)

df

Segmentation of Machines Idle Time

In [ ]:
list_test = [eval(x) for x in df['Machines Idle Time'].tolist()]

for i in range(max_length):
    col_name = f'Machine_{i+1}_IT'
    df[col_name] = [lst[i] if i < len(lst) else 0 for lst in list_test]

df.drop(columns=['Machines Idle Time'], inplace=True)

Segmentation of Machines CT

In [ ]:
list_test = [eval(x) for x in df['Machine CT'].tolist()]

for i in range(max_length):
    col_name = f'Machine_{i+1}_CT'
    df[col_name] = [lst[i] if i < len(lst) else 0 for lst in list_test]

df.drop(columns=['Machine CT'], inplace=True)

Segmentation of Machines Breakdowns

In [ ]:
list_test = [eval(x) for x in df['Machines Breakdowns'].tolist()]

for i in range(max_length):
    col_name = f'Machine_{i+1}_BD'
    df[col_name] = [lst[i] if i < len(lst) else 0 for lst in list_test]

df.drop(columns=['Machines Breakdowns'], inplace=True)

Segmentation of Buffers States

In [ ]:
list_test = [eval(x) for x in df['Buffers State'].tolist()]

simple_arrays = [[item for tup in inner_list for item in tup] for inner_list in list_test]

for i in range(max_length*2):
    buffer_number = (i // 2) + 1
    buffer_subscript = ".1" if i % 2 == 0 else ".2"
    col_name = f'Buff_{buffer_number}{buffer_subscript}_ST'
    df[col_name] = [lst[i] if i < len(lst) else 0 for lst in simple_arrays]


df.drop(columns=['Buffers State'], inplace=True)

Segmentation of Machines State

In [ ]:
list_test = [eval(x) for x in df['Machines State'].tolist()]
modified_array = [[1 if element else 0 for element in row] for row in list_test]

for i in range(max_length):
    col_name = f'Machine_{i+1}_State'
    df[col_name] = [lst[i] if i < len(lst) else 0 for lst in modified_array]

df.drop(columns=['Machines State'], inplace=True)

Segmentation of Robot State

In [ ]:
data_list = [eval(x) for x in df['Robot State'].tolist()]


df1 = pd.DataFrame(data_list, columns=['From', 'To'])

elements = [f"M{i}" for i in range(1, 21)]

df1_encoded = pd.DataFrame(0, columns=elements, index=df1.index)

for index, row in df1.iterrows():
    df1_encoded.loc[index, row['From']] = 1
    df1_encoded.loc[index, row['To']] = 1

df1_encoded['InputStock'] = df1_encoded['InputStock'].fillna(0).astype(int)
df1_encoded['OutputStock'] = df1_encoded['OutputStock'].fillna(0).astype(int)

df1.drop(['From', 'To'], axis=1, inplace=True)

df1 = pd.concat([df1, df1_encoded], axis=1)

df = df.join(df1)

df.drop(columns=['Robot State'], inplace=True)

Adding Machine Operating Time column

In [ ]:

columns = [f'Buffer {i} cap' for i in range(1, 21)] + [f'Machine {i} OperatingT' for i in range(1, 21)]
data = np.zeros((343, 40))

data[:, :7] = 1

data[:, 20:27] = np.array([[100, 120, 200, 50, 20, 100, 50]])

df2 = pd.DataFrame(data, columns=columns)

df =df.join(df2)

#scaler = MinMaxScaler()

#df_normalized_values = scaler.fit_transform(df.values)
#df = pd.DataFrame(df_normalized_values, columns=df.columns, index=df.index)

#df.drop(columns=['Sim Instant'], inplace=True)
pd.set_option('display.max_columns', None)

df

Calculating Colsest Index to 1h of Sim Instant

In [ ]:
first_row_index = df[df['Sim Instant'] >= 3600].index.min()

diff1 = abs(df.at[first_row_index, 'Sim Instant'] - 3600)
diff2 = abs(df.at[first_row_index - 1, 'Sim Instant'] - 3600)

closest_row_index = first_row_index if diff1 < diff2 else first_row_index - 1

print("Index of the closest row to 3600:", closest_row_index)


In [ ]:
# step = closest_row_index + 1 
# X = [[] for _ in range(182)]
# y = []

# for i in range(0, df.shape[0] - step):
#     for j in range(182):
#         X[j].append(df.iloc[i:i + step, j])
#     y.append(df.iloc[i + step, 0])

Creating the new dataframe

In [ ]:
new_df = df.loc[:, 'Machine_1_IT':'Machine_20_IT']
new_df = pd.concat([new_df, df.loc[:, 'M1':'M20']], axis=1)
new_df = pd.concat([new_df, df[['InputStock', 'OutputStock']]], axis=1)
new_df = pd.concat([new_df, df.loc[:, 'Buff_1.1_ST':'Buff_20.2_ST']], axis=1)

new_df['Machine_IT_Total'] = new_df.loc[:, 'Machine_1_IT':'Machine_20_IT'].max(axis=1)

new_df.drop(columns=new_df.loc[:, 'Machine_1_IT':'Machine_20_IT'].columns, inplace=True)
new_df

Preparing the datya for the model

In [ ]:
steps = 1
X = [[] for _ in range(63)]
y = []

for i in range(new_df.shape[0] - steps):
    for j in range(63):
        X[j].append(new_df.iloc[i:i + steps, j])
    y.append(new_df.iloc[i + steps, 62])

In [ ]:
y = np.array(y)
for i in range(len(X)):
    X[i] = np.array(X[i])



y=np.reshape(y, (len(y),1))
y.shape

In [ ]:
X = np.stack([X[i] for i in range(63)], axis=2)

In [ ]:
X.shape

In [ ]:
X_train, X_test = X[:-42], X[-42:]
y_train, y_test = y[:-42], y[-42:]
X_train.shape[1], X_train.shape[2]

In [ ]:
X_train[10]

In [ ]:
X_train[11]

In [ ]:
y_test

In [ ]:
X_train.shape

Creating the model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras import optimizers

In [ ]:
model = Sequential()
# set return requesnce = true, if we are going to have another LSTM layer
model.add(LSTM(10, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(LSTM(10, return_sequences=False))
model.add(Dense(10, activation='relu'))
model.add(Dense(1))

In [ ]:
model.summary()

In [ ]:
#from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

filepath = 'models/{epoch:02d}-{loss:.4f}-{val_loss:.4f}-{mae:.4f}-{val_mae:.4f}.hdf5'

# callbacks = [EarlyStopping(monitor='val_loss', patience=20),
#              ModelCheckpoint(filepath, monitor='loss', save_best_only=True, mode='min')]

In [ ]:
model.compile(optimizer='ADAM', loss='mse', metrics=['mae'])

Trainning the model

In [ ]:
model.fit(X_train, y_train, validation_split=0.2, epochs=1000, batch_size=1)
#callbacks=callbacks,

In [ ]:
MSE, MAE = model.evaluate(X_test, y_test)
print(MSE, MAE)

Prediction

In [ ]:
predictions = model.predict(X_test)
predictions

In [ ]:
y_test

Ploting the results

In [ ]:
plt.style.use('fivethirtyeight')
plt.figure(figsize=(24,12))
plt.plot(predictions)
plt.plot(y_test)
plt.legend(['Forecast', 'Actual'])
plt.show()

Print a desired row from the df

In [ ]:
desired_value = 300
result_df = df.loc[df.index == desired_value]
result_df

Other random models

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Assuming df is your DataFrame with normalized data

# Convert the DataFrame to a numpy array
data = df.values

# Split the data into input (X) and output (y)
X = data[:-1, :]  # All rows except the last one
y = data[1:, :]   # All rows except the first one

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Reshape the data for LSTM input (samples, time steps, features)
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

# Build the LSTM model
model = Sequential()
model.add(LSTM(50, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(y_train.shape[1]))  # Output layer with the same number of features as y

# Compile the model
model.compile(optimizer='adam', loss='mse')  # You can use a different optimizer or loss function if needed

# Train the model
model.fit(X_train, y_train, epochs=100, batch_size=64, validation_data=(X_test, y_test))

# Evaluate the model
loss = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss}")

# Now you can use the trained model to predict the next row
# For example, if you want to predict the next row after the last row in your training data:
last_row = X_train[-1, :, :].reshape((1, 1, X_train.shape[2]))
predicted_row = model.predict(last_row)


# Assuming predicted_row is your numpy array
predicted_df = pd.DataFrame(predicted_row.reshape((1, -1)))

# Display the DataFrame
predicted_df

In [ ]:
import matplotlib.pyplot as plt

# Assuming df is your original DataFrame
# Extract the actual values for Machine_1_T from the test set
actual_values = y_test[:, df.columns.get_loc('Machine_1_IT')]

# Predict the next values using the trained model
predicted_values = model.predict(X_test)[:, df.columns.get_loc('Machine_1_IT')]

# Create a DataFrame for visualization
visualization_df = pd.DataFrame({'Actual': actual_values, 'Predicted': predicted_values})

# Plot the actual vs predicted values
plt.figure(figsize=(10, 6))
plt.plot(visualization_df['Actual'], label='Actual', marker='o')
plt.plot(visualization_df['Predicted'], label='Predicted', marker='x')
plt.title('Actual vs Predicted Values for Machine_1_IT')
plt.xlabel('Data Points')
plt.ylabel('Values')
plt.legend()
plt.show()
predicted_values.size

In [ ]:
# Assuming predicted_row is your numpy array
predicted_df = pd.DataFrame(predicted_row.reshape((1, -1)))

# Display the DataFrame
predicted_df

In [ ]:
import pandas as pd
import numpy as np

# Assuming you have a DataFrame named 'df' with 182 columns and 343 rows

# Creating a sample DataFrame for illustration
# Replace this with your actual DataFrame
data = np.random.randn(343, 182)
df = pd.DataFrame(data, columns=[f'feature_{i}' for i in range(1, 183)])

# Setting a sequence length (you can adjust this based on your requirements)
seq_len = 10

# Reshape the DataFrame into sequences
num_sequences = len(df) // seq_len
df_reshaped = df.iloc[:num_sequences * seq_len, :].values.reshape(num_sequences, seq_len, -1)

# 'num_sequences' is the number of sequences, 'seq_len' is the length of each sequence,
# and '-1' in the reshape means the remaining dimension is inferred based on the other dimensions.

# Now 'df_reshaped' is a 3D tensor suitable for input into an LSTM
print("Original DataFrame shape:", df.shape)
print("Reshaped DataFrame shape:", df_reshaped.shape)
df_reshaped